# Gibbs sampling implementation - HOANG Hai Nam

## Pre-requesites

In [1]:
import random as rd
import numpy as np

NUC_TO_NUM = {'A': 0, 'C': 1, 'G': 2, 'T': 3}


In [15]:
def hamming_distance(s1, s2):
    """
    Number of differences between two sequences
    :param s1: str
    :param s2: str
    :return: int, number of differences
    """
    return sum([int(s1[i] != s2[i]) for i in range(len(s1))])

The hamming_distance function calculates the differences between 2 sequences, in other words how many changes to one sequence needed to be identical with the other sequence.

In [16]:
def score_motif(motifs):
    """
    Will build the most common kmer among all the motifs and then compute the Hamming Distance from this sequence
    :param motifs: list of str, DNA motifs of the same size
    :return: int, score of the collection of motifs
    """
    score = 0
    common_kmer = [{'A': 0, 'C': 0, 'T': 0, 'G': 0} for _ in range(len(motifs[0]))]
    for motif in motifs:
        for nuc in range(len(motif)):
            common_kmer[nuc][motif[nuc]] += 1
    common_kmer = "".join([sorted(nucleotide_dict.items(), key=lambda item: item[1], reverse=True)[0][0]
                           for nucleotide_dict in
                           common_kmer])  # take the most frequent nucleotide at each position and join

    for motif in motifs:
        score += hamming_distance(motif, common_kmer)
    return score

The score_motif function recieves a list of motifs, builds the most common kmer, and then computes the Hamming Distance between this common kmer and the motifs. The lower the score in the return, the better it is as the closer the motifs are together. The score is calculated as the sum of hamming distances from each motif to the common kmer/motif.

In [17]:
def build_profile(motifs):
    """
    Build a profile motif based on a collection of motifs
    :param motifs: list of str, DNA motifs of the same size
    :return: matrix of floats of size 4 x size of a motif, probability of finding each nucleotide at each position of
     of the motif
    """
    profile = [[1.0 for _ in range(len(motifs[0]))] for _ in range(4)]  # pseudocount

    # count the nucleotide at each position for each motif
    for motif in motifs:
        for i in range(len(motif)):
            nucleotide = motif[i]
            profile[NUC_TO_NUM[nucleotide]][i] += 1

    for line in range(4):
        for col in range(len(profile[0])):
            profile[line][col] /= len(motifs)  # from count to frequency

    return profile

This function recieves a set of motifs and builds a probability profile in the form of a matrix, based on this profile we can calculate the probability of a certain sequence(or motif) of the motif length. The 4 rows represent the 4 different nucleotides. and the columns represent the positions of nucleotide in the kmer. This function loops from position to position of kmer length, counting how many of what nucleotides appear at each position, then the results are converted into probabilities by dividing with the length, all the probabilities of nucleotides at each position are then intergrated into a matrix.

In [18]:
def build_kmer_based_profile(sequence, profile, k):
    """
    Produce the k-mer in a sequence with their probability based on a given profile
    :param sequence: str, DNA sequence
    :param profile: matrix of floats of size 4 x n, probability of finding each nucleotide at each position j, 0<=j<n
    :param k: size of the k-mer desired (normally n, as same size as the profile)
    :return: str, k-mer selected randomly according to the distribution of the probabilities the k-mers came
    from the profile
    """
    kmers = []
    plist = []
    i = 0
    e = k
    while e <= len(sequence):
        kmers.append(sequence[i:e])
        i += 1
        e += 1

    for x in range(len(kmers)): #loops from kmers to kmers
        plist.append(1)
        for z in range(len(kmers[x])): #loops from position to position
            if kmers[x][z] == 'A':
                plist[x] *= profile[0][z]
            elif kmers[x][z] == 'C':
                plist[x] *= profile[1][z]
            elif kmers[x][z] == 'G':
                plist[x] *= profile[2][z]
            elif kmers[x][z] == 'T':
                plist[x] *= profile[3][z]

    minprob = np.min([i for i in plist if i != 0]) # minimal non-zero probability, just in case a zero arises
    for i in range(len(plist)): # dividing entire list by lowest probabily to convert them to ratios
        plist[i] /= minprob
    sumrat = sum(plist)

    for i in range(len(plist)): # dividing entirelist by its sum to convert them into probabilities
        plist[i] /= sumrat

    return np.random.choice(kmers, p=plist) # weighted random kmer is chosen


This function takes a sequence and a probability profile, breaks the sequence into kmers of a given size(has to be the same length as the one associated with the profile matrix). For each kmer, a probability is calculated with the probability profile matrix, all the probabilities are then appended to a list, this list is then treated by dividing entire list by lowest value to obtain the probability ratio, then this list is divided again by the sum of all the ratios. Finally a list of probabilities of each kmer is obtained, using the numpy's random choice function, this probability list is used as weights to choose between the kmers. The higher the probability, the more likely a kmer is chosen. This is done to avoid falling into the greedy approach when only the most probable is chosen, local scores are prioritised, as such the final global score may not be optimal as substantially less routes are considered. Choosing a poor local score at one instance can open up other possible routes later that could lead to better global score, as such this random choice is made to be flexible, while still leading a to better scoring motif.

In [19]:
def gibbs_sampling(sequences, k, number_iterations):
    """
    Find the best motifs in the sequences by starting with random starting positions for the k-mer in each sequences
    and then iterating by excluding a motif among them, building a profile matrix based on the remaining
    motifs and replacing the motif excluded by the most probable k-mer motif based on the profile. We keep as best
    motifs the motifs which obtained the lowest score until now.
    :param sequences: list of str, DNA sequences where we want to find the best k-mers
    :param k: int, size of the motif desired
    :param number_iterations: int, number of time we do the operation of excluding, building a profile and replacing
    the motif excluded with the most probable k-mer of the sequence where the excluded motif came from based on the
    built profile
    :return: tuple(list of str, int), (best motifs obtained, score of the list of motifs)
    """
    kmers = []
    for s in range(len(sequences)):
        i = 0
        e = k
        kmers.append([])
        while e <= len(sequences[s]):
            kmers[s].append(sequences[s][i:e])
            i += 1
            e += 1
    motifs = []
    for seq in kmers:
        motifs.append(rd.choice(seq))
    topmotifs = list(motifs)
    topscore = score_motif(topmotifs)
    for j in range(0, number_iterations - 1):
        i = rd.randint(0, len(sequences) - 1)
        del motifs[i]
        pf = build_profile(motifs)
        motifs.insert(i,build_kmer_based_profile(sequences[i], pf, k))
        if score_motif(motifs) < topscore:
            topmotifs = list(motifs)
            topscore = score_motif(topmotifs)
    return topmotifs, topscore

Similar to the build_kmer_based_profile() function, first kmers are generated from the sequences, each sequence will then have a list of kmers associated to them. Then from each of these lists, a random kmer/motif is chosen. At the end of this step we'll have one kmer associated with each sequence. These will be our motif set, from this motif set, a motif is chosen at random to be excluded and replaced later, a profile matrix is then build from the remaining motifs. Using this profile matrix, a motif is chosen at random from the sequence that's associated with the removed motif. The excluded motif will then be replaced with a (likely) better motif. The final global score of the set of motifs are then calculated at the end, if the score is better with replacement, the motif set is replaced with the new one. Each iteration a new motif is then chosen to be potentially replaced. This is repeated for number_iterations of times. The function then returns the motif set with the best score, and its score, as a 2-tuple.

In [29]:
def main(sequences, k, number_iterations, restarts):
    """
    Program where we will call a number of times the Gibbs sampler to reach the best motifs possibles
    :param sequences: list of str, DNA sequences where we want to find the best k-mers
    :param k: int, size of the motif desired
    :param number_iterations: int, number of time we do the operation of excluding, building a profile and replacing
    the motif excluded with the most probable k-mer of the sequence where the excluded motif came from based on the
    built profile
    :param restarts: int, number of times we repeat the Gibbs sampler operation
    :return: list of str, best motifs obtained
    """
    topmotifs = gibbs_sampling(sequences, k, number_iterations)[0]
    for j in range(0, restarts-1):
        motifs = gibbs_sampling(sequences, k, number_iterations)[0]
        if score_motif(motifs) < score_motif(topmotifs):
            topmotifs = list(motifs)
    return topmotifs


This function follow similar principles to the previous, in that it repeats the previous function multiple times and each time it'll check for score improvements. As the start of gibbs_sampling(), we choose a random motif/kmer from each sequences, the whole of gibbs_sampling() function needs to be repeated in order to consider most if not all possibilities. The final return will be the motifs that has the best score(lower is better). The Gibbs_sampling() function is repeated 'restarts' number of times.

## Inputs

In [28]:
SEQUENCES = [
    "CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA",
    "GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG",
    "TAGTACCGAGACCGAAAGAAGTATACAGGCGT",
    "TAGATCAAGTTTCAGGTGCACGTCGGTGAACC",
    "AATCCACCAGCTCCACGTGCAATGTTGGCCTA"]
K = 8
NUMBER_ITERATIONS = 20
RESTARTS = 300

top_motif = main(SEQUENCES, K, NUMBER_ITERATIONS, RESTARTS)
print(top_motif)
print("Score:", score_motif(top_motif))

['TCTCGGGG', 'CCAAGGTG', 'TACAGGCG', 'TTCAGGTG', 'TCCACGTG']
Score: 9


## Ideal output

In [22]:
#the 'best' motifs im supposed to get after running above code
print(['TCTCGGGG', 'CCAAGGTG', 'TACAGGCG', 'TTCAGGTG', 'TCCACGTG'])


['TCTCGGGG', 'CCAAGGTG', 'TACAGGCG', 'TTCAGGTG', 'TCCACGTG']
